In [1]:
# importing the Pytorch library, which is a ml framework to create and train neural net
import torch
# importing the nn module which provides a variety of tools and functions for building nn
from torch import nn
# importing DataLoader class from torch.utils.data module, which is a utility for efficiently loading and processing data in batches during model training
from torch.utils.data import DataLoader
# importing datasets module from torchvision package for computer vision tasks. It provides us with pre-loaded datasets such as CIFAR-10, MNIST, and ImageNet
from torchvision import datasets
# Importing ToTensor class from transforms module of torchvision. This class converts PIL images or numpy arrays into PyTorch tensors, which are the fundamental data structure for PyTorch computations.
from torchvision.transforms import ToTensor

C:\Users\Fatihi\.conda\envs\Pytorch_1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Every TorchVision `Dataset` includes two arguments: transform and target_transform to modify the samples and labels respectively.
### We download the FashionMNIST dataset from open datasets and creates two datasets, one for training one for testing, each with their own arguments that define how the data should be handled.
training_data = datasets.FashionMNIST(
root = "data",
train = True,
download = True,
transform = ToTensor(),
)
'''
root = "data" : specifies the directory where the data will be downloaded and saved
train = True : indicates that we want to download the training data
download = True : means that if the data isn't already downloaded, it will be downloaded from the interwebz
transform = ToTensor() : transforms the image data into PyTorch tensors
'''

test_data = datasets.FashionMNIST(
root = "data",
train = False,
download = True,
transform = ToTensor(),
)
'''
train = True : downloading the test data
'''

'\ntrain = True : downloading the test data\n'

In [3]:
### We pass the Dataset as an argument to dataLoader, which wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading

# We set the batch size to 64, which means that each iteration of the DataLoader will return 64 samples from the dataset
batch_size = 64 

# We create a DataLoader for the training data, which takes in the `training_data` object and the `batch_size` parameter and returns a generator that can be used to iterate over batches of data during training.
train_dataloader = DataLoader(training_data, batch_size = batch_size)

# We create a DataLoader for the test data, which is similar tot the training data DataLoader. The difference is that this DataLoader is used to iterate over batches of data during testing.
test_dataloader = DataLoader(test_data, batch_size = batch_size)

# We start a loop that iterates over batches of data from the test DataLoader. the `X` var contains the input data for the batch, and `y` contains the corresponding labels
for X, y in test_dataloader:
    # We print the shape of the input data for the current batch. The format of the shape is [N,C,H,W] where N is the batch size, C is the number of channels in the input data, H is the height of the input data, and W is the width of the input data
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    # We print the shape and data type of the labels for the current batch. The shape is simply the batch size, and data type is printed aswell.
    print(f"Shape of y : {y.shape} {y.dtype}")
    # We break out of the loop after the first iteration, which is useful for testing and debugging purposes to check the shape and data type of the first batch of data.
    break



Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64


In [5]:
# CREATING THE MODEL

# To define a nn in PyTorch, we create a class that inherits from nn.Module.
# We define the layers of the network in the __init__ fct and specify how data will pass through the network in  the `forward` function.
# To accelerate operations in the nn, we move it to the GPU if available.

# We check if CUDA-enabled GPU is available, otherwise we checks if an MPS-enabled GPU is available, if none is available we use cpu
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
#device = "cpu"
print(f"Using {device} device")

#Defining the model, it defines `NeuralNetwork` class that inherits from `nn.Module`
class NeuralNetwork(nn.Module):
    # Within the class, the `__init__` method initializes the model by defining two layers `nn.Flatten` and `nn.Sequential`
    def __init__(self):
        super().__init__()
        # The flatten layer fattens the input tensor into a 1D tensor
        self.flatten = nn.Flatten()
        # The sequential layer defines a fully connected neural network with three linear layers and two ReLU activations
        '''
        The neural network consists of 3 fully connected layers r linear transformations ofollowed by activation fcts (ReLU) applied element-wise to the outputs of the linear transformations.
        The architecture of the nn consists of:
        - input layer : the input of the network is a tensor of shape [batch_size, 1, 28, 28], where the batch_size is the number of samples in a batch. The input tensor is first fattened into a vector of size 28*28 = 784 using nn.Flatten()
        - Hidden layer 1 : The first linear transformation takes the fattened input of size 784 and outputs a tensor of size 512. This transformation is performed using nn.Linear(28*28, 512).
        - ReLU activation 1 : A rectified linear unit (ReLU) activation function is applied to the output of the first linear transformation using nn.ReLU().
        - Hidden layer 2 : the second linear transformation takes the output of the first activation function of size 512 and outputs another tensor of size 512. This transformation is performed using nn.Linear(512, 512)
        - ReLU activation 2 : Another ReLU activation function is applied to the output of the second linear transformation using nn.ReLU()
        - Output layer: The third and final linear transformation takes the output of the second activation function of size 512 and outputs a tensor of size 10 which represents the 10 classes of the FashionMNIST dataset. This transformation is performed using nn.Linear(512, 10)
        Therefore, the specific numbers were chosen to provide enough capacity for the network to learn the complex patterns in the FashionMNIST dataset, while preventing overfitting. These numbers were likey deterined through experimentation and tuning.
        '''
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    # The forward method specifies how data flow through the network
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
# OPTIMIZING THE MODEL PARAMETERS


#To train a model, we need a loss function and an optimizer

# We create an instance of the `CrossEntropyLoss` class from thee `nn` module. This loss function will be used to compute the difference between the predicted outputs and the actual labels.
loss_fn = nn.CrossEntropyLoss()
# We create an instance of the `SGD`(Stochastic Gradient Descent) optimizer from the `torch.optim` module. It takes as input the `parameters` of the `model` and a learning rate of `1e-3`, which determines the step size at each iteration durin the optimization process.
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# We defines a function called `train` that takes in a `dataloader`, `model`, `loss_fn`, and `optimizer` as args 
def train(dataloader, model, loss_fn, optimizer):
    # We compute the total size of the dataset by getting the length of the `dataset` object associated with the input `dataloader`
    size = len(dataloader.dataset)
    # We set the model to training mode, which enables certain features such as dropout and batch normalization
    model.train()
    # We iterate over the batches of data in the input `dataloader`, where `batch` is the index of the current batch and `(X, y)` is a tuple containing the input data and the corresponding labels
    for batch, (X, y) in enumerate(dataloader):
        # We move the input data and labels to the specified device(either CPU or GPU)
        X, y = X.to(device), y.to(device)
        
        ### Compute prediction error
        
        # We compute the predicted outputs by passing the input data through the `model`
        pred = model(X)
        # We compute the loss by comparing the predicted outputs to the actual labels using the input `loss_fn`
        loss = loss_fn(pred, y)
        
        ### Backpropagation
        
        # We reset the gradient of all the parameters in the `model` to zero.
        optimizer.zero_grad()
        # We compute the gradients of the parameters with respect to the loss
        loss.backward()
        # We update the parameters using the gradients computed in the previous step
        optimizer.step()
        
        # We print out the current loss and the progress of the training process every 100 batches. The `loss.item()` method extract the scalar value of the loss from the tensor, and `current` computes the number of example that have been processed so far.
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss {loss:>7f} [{current:>5d}{size:>5d}]")

In [7]:
# Checking model's performance against the test dataset to ensure it is learning

# We define the test function that has 3 args (dataloader, a model, and a loss fct), this fct is used to evaluate the performance of the model on a given dataset
def test(dataloader, model, loss_fn):
    # We calculate the total number of data samples in the dataloader
    size = len(dataloader.dataset)
    # We calculate the total number of batches in the dataloader
    num_batches = len(dataloader)
    # We set the model to evaluation mode, which is eimportant because some layers of the model, such as Dropout and BatchNorm, behave differently during training and evaluation
    model.eval()
    # We initialize 2 vars to 0, `test_loss` and `correct`, which are used to accumulate the loss and accuracy values as the model is evaluated on the test dataset
    test_loss, correct = 0, 0
    # We create a context where no gradient claculations will be made, because we don't need to calculate gradients during model evaluation, and calculating gradients can slow down the evaluation process
    with torch.no_grad():
        # We iterates over the batches in the dataloader, and for each batch, it extracts the input tensor X and the label tensor y
        for X, y in dataloader:
            # We move the input tensor X and label tensor y to the device(cpu)
            X, y = X.to(device), y.to(device)
            # We use the model to make a prediction on the input tensor X
            pred = model(X)
            # We calculate the loss between the predicted values and the true labels, and adds it to the test_loss var
            test_loss += loss_fn(pred, y).item()
            # We calculate the number of correctly classified samples in the batch, and add it to the correct var
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    # We calculate the average test loss over all batches
    test_loss /= num_batches
    # We calculate the overall accuracy of the model on the entire test dataset
    correct /= size
    
    # Print the final evaluation metrics: the test accuracy and the average test loss
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>f} \n")
    
    

In [8]:
'''
The training process is conducted over several iterations(epochs). 
During each epoch, the model learns parameters to make better predictions. 
We print the model's accuracy and loss at each epoch; 
we'd like to see the accuracy increase and the loss decrease with every epoch
'''
# we set the number of epochs to 5
epochs = 5
# We start a loop that execute for the number of epochs specified previously
for t in range(epochs):
    # We print a header indicating the current epoch number, which will be used to track the progress of the training process
    print(f"Epoch {t+1}\n-------------------------------")
    # We call the `train()` fct to train the `model` on the `train_dataloader`, using the specifed `loss_fn` and `optimizer`. This fct updates the parameters of the `model` based on the loss computed during training
    train(train_dataloader, model, loss_fn, optimizer)
    # We call the `test()` fct to test the `model` on the `test_dataloader`, usin the same `loss_fn`. This function computes the accuracy and average loss of the `model` on the test data
#Baam!        
print("Done!")

'''
The numbers below represent the loss values at different epochs during the trainin fo a machine learning model.
The loss is a measure of how well the model is performing on the training data.
It represets the diffrence between the predicted output of the model and the actual output(target) for a given input.
In this case, the model was trained for 5 epochs and the loss values were recorded at different steps of each epoch.
The first column represents the loss value, and the number in brackets represents the step number(that is, the number of samples processed during the training).
The loss values decrease over time, *which means the model is improving and becoming better at predicting the target output.*
The training is considered complete after the last epoch.

"
Increasing the number of eposhs is not always the best approach. If the model is already starting to overti the training data, increasing the number of epochs may cause the model to perform worse on new unseen data(test set).
So imay be better to introduce regularization techniques or adjust the model architecture to prevent overfitting.

Also, some models may reach a point of diminishing returns where further training epochs do not significantly imporove performance on the test set.
Therefore, it's important to monitor the model's performance on the validation set and be mindful of when to stop training to prevent overfitting
"

"
Regularization Techniques:
- L1 and L2 regularization: These techniques add a penalty term to the loss function that encourages the model to use smaller weights.
L1 regularization (Lasso) encourages sparse weights, meaning that some weights will be set to zero, while L2 regularization(Ridge) encourages smaller weights but deos not result in sparse weights.
- Dropout: Droptout randomly removes neurons from the neural network during training. This technique can help prevent the network from relying too heavily on any one neuron or set of neurons and encourages the netweork to learn more rubust features.
- Early Stopping: This technique involves monitoring the validation loss during training and stopping the training when the validation loss starts to increase. This can help prevent the model from overfitting the training data and generalize better to new, unseen data.
- Batch normalization: This technique involves normalizing the inputs to each layer in the network, which can help prevent the values from becoming too large or too small during training.
This can help the model learn more quickly and generalize better.
- Data augmentation: This technique involves applying random transformations to the training data(such as rotaion, scaling, or flipping) to create new, slightly different training examples. This can help prevent overfitting and improve the model's ability to generalize to new data.

These regularization techniques can be applied separately or in combination with each other, depending on the specific problem and model architecture. It's important to experiment with different techniques to find the optimal combination for your model.
"
'''

Epoch 1
-------------------------------
loss 2.312799 [   6460000]
loss 2.305877 [ 646460000]
loss 2.281796 [1286460000]
loss 2.269885 [1926460000]
loss 2.253437 [2566460000]
loss 2.218624 [3206460000]
loss 2.231571 [3846460000]
loss 2.193518 [4486460000]
loss 2.195632 [5126460000]
loss 2.156775 [5766460000]
Epoch 2
-------------------------------
loss 2.168800 [   6460000]
loss 2.161884 [ 646460000]
loss 2.099108 [1286460000]
loss 2.116611 [1926460000]
loss 2.061450 [2566460000]
loss 1.997741 [3206460000]
loss 2.032485 [3846460000]
loss 1.942785 [4486460000]
loss 1.959882 [5126460000]
loss 1.884616 [5766460000]
Epoch 3
-------------------------------
loss 1.914988 [   6460000]
loss 1.886990 [ 646460000]
loss 1.765134 [1286460000]
loss 1.813632 [1926460000]
loss 1.702115 [2566460000]
loss 1.650701 [3206460000]
loss 1.684521 [3846460000]
loss 1.573523 [4486460000]
loss 1.611792 [5126460000]
loss 1.510683 [5766460000]
Epoch 4
-------------------------------
loss 1.586871 [   6460000]
los

'\nThe numbers below represent the loss values at different epochs during the trainin fo a machine learning model.\nThe loss is a measure of how well the model is performing on the training data.\nIt represets the diffrence between the predicted output of the model and the actual output(target) for a given input.\nIn this case, the model was trained for 5 epochs and the loss values were recorded at different steps of each epoch.\nThe first column represents the loss value, and the number in brackets represents the step number(that is, the number of samples processed during the training).\nThe loss values decrease over time, *which means the model is improving and becoming better at predicting the target output.*\nThe training is considered complete after the last epoch.\n\n"\nIncreasing the number of eposhs is not always the best approach. If the model is already starting to overti the training data, increasing the number of epochs may cause the model to perform worse on new unseen dat

In [9]:
# SAVING THE MODEL

# fct that saves the learned parameters of a PyTorch model to a file
# Specifically it saves the state dictionary of the model, which is a Py dict object containing all the learnable parameters of the model, such as the weights and biases of each layer.
torch.save(model.state_dict(), "model.pth")

print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [10]:
# LOADING THE MODEL

# We create an instance of a PyTorch neural network model
model = NeuralNetwork()

# We  load the learned parameters of the model from the file and assigns them to the corresponding layers of the `model` instance
# This fct is commonly used to load a pre-trained model so that it can be used for inference or fine-tuning
# The loaded state dict should match the structure of the model instance, otherwise an error will occur.
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [11]:
# We define a list of 10 classes that the model is trained to recognize. These correspond to different types of clothing items.
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

# We set the model to evaluation mode, which is important because some layers in the model, such as dropout and batch normalization, behave differently during training and evaluation.
model.eval()

# We assign the first image and its corresponding label form the test dataset to the variable `x` and `y` respectively.
x, y = test_data[0][0], test_data[0][1]

# torch.no_grad() creates a context where any operations performed within it won't be tracked for automatic differentiation, which can save memory and computation time.
with torch.no_grad():
    # We pass the image `x` through the pre-trained model to obtain a prediction
    pred = model(x)
    # We use the predicted class index and the true class index to loop up the corresponding class names from the `classes` list
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
